In [1]:
from datasets import load_dataset, Dataset
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig
from trl import SFTConfig, SFTTrainer

In [2]:
dataset = load_dataset('iamjoon/klue-mrc-ko-rag-dataset', split='train')
print(set(dataset['type']))
dataset = dataset.class_encode_column('type')

{'synthetic_question', 'mrc_question', 'mrc_question_with_1_to_4_negative', 'paraphrased_question', 'no_answer'}


In [3]:
# cast_column 메서드 활용도 가능
type_name_dict = {1:'mrc_question_with_1_to_4_negative', 3:'paraphrased_question', 2:'no_answer', 4:'synthetic_question', 0:'mrc_question'}

In [4]:
system_message = '''
당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.

다음의 지시사항을 따르십시오.
1. 질문과 검색 결과를 바탕으로 답변하십시오.
2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.
3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문에 대한 내용이 없습니다." 라고 답변하십시오.
4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.
5. 예를 들어 특정 문장이나 문단을 1번 문서와 5번 문서에서 동시에 인용했다면 뒤에 [[ref1]], [[ref5]]라고 기재하십시오.
6. 최대한 다수의 문서를 인용하여 답변하십시오.

검색 결과:
----------
{search_result}'''

In [5]:
print('원본 데이터의 type 분포:')
for type_name in set(dataset['type']):
    print(f'{type_name_dict[type_name]}: {dataset['type'].count(type_name)}')

원본 데이터의 type 분포:
mrc_question: 491
mrc_question_with_1_to_4_negative: 296
no_answer: 404
paraphrased_question: 196
synthetic_question: 497


In [6]:
split_dataset = dataset.train_test_split(test_size=0.5, stratify_by_column='type')
train_dataset_format = split_dataset['train']
test_dataset_format = split_dataset['test']

In [7]:
def format_data(sample):
    search_result = '\n------\n'.join(f'문서{idx+1}: {result}' for idx, result in enumerate(sample['search_result']))

    return {
        'messages':[{'role':'system', 'content':system_message.format(search_result=search_result)},
                    {'role':'user', 'content':sample['question']},
                    {'role':'assistant', 'content':sample['answer']}]
    }

In [8]:
train_dataset = [format_data(train_data) for train_data in train_dataset_format]
test_dataset = [format_data(test_data) for test_data in test_dataset_format]

In [9]:
print(f'\n전체 데이터 분할 결과: Train {len(train_dataset)}개, Test {len(test_dataset)}개')
print('--'*20)
print('\n학습 데이터의 type 분포:')
for type_name in set(dataset['type']):
    print(f'{type_name_dict[type_name]}: {train_dataset_format['type'].count(type_name)}')
print('\n테스트 데이터의 type 분포:')
for type_name in set(dataset['type']):
    print(f'{type_name_dict[type_name]}: {test_dataset_format['type'].count(type_name)}')


전체 데이터 분할 결과: Train 942개, Test 942개
----------------------------------------

학습 데이터의 type 분포:
mrc_question: 245
mrc_question_with_1_to_4_negative: 148
no_answer: 202
paraphrased_question: 98
synthetic_question: 249

테스트 데이터의 type 분포:
mrc_question: 246
mrc_question_with_1_to_4_negative: 148
no_answer: 202
paraphrased_question: 98
synthetic_question: 248


In [10]:
train_dataset[345]['messages']

[{'role': 'system',
  'content': '\n당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.\n\n다음의 지시사항을 따르십시오.\n1. 질문과 검색 결과를 바탕으로 답변하십시오.\n2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.\n3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문에 대한 내용이 없습니다." 라고 답변하십시오.\n4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.\n5. 예를 들어 특정 문장이나 문단을 1번 문서와 5번 문서에서 동시에 인용했다면 뒤에 [[ref1]], [[ref5]]라고 기재하십시오.\n6. 최대한 다수의 문서를 인용하여 답변하십시오.\n\n검색 결과:\n----------\n문서1: 주요 유통 및 필수 소비재 업체들이 여기저기서 허리띠를 졸라매고 있다. 무분별하게 ‘덩치’를 키우기보다 수익을 많이 남길 수 있는 상품의 비중을 높이고, 돈이 안 되는 브랜드들을 접는 등 사업구조 개편에 힘을 쏟고 있다. 21일 유통업계에 따르면 신세계백화점은 온라인 쇼핑몰인 신세계몰의 수익성을 개선하는 데 주력키로 했다. 특정 상품을 살 때 사용하면 정상가보다 15~20% 할인해줬던 할인 쿠폰 발행도 대폭 줄였다. 또 신세계몰에서 판매하는 상품 중 백화점에도 입점해 있는 브랜드 비중을 30%대에서 40%대로 높였다. 백화점에 입점돼 있는 브랜드는 유통단계가 축소돼 다른 상품보다 이익률이 상대적으로 높다. 신세계몰의 지난달 영업손실은 16억원으로, 지난해 같은 달보다 36% 감소했다. 확장을 자제하면서 같은 기간 매출은 279억원으로, 전년 동기 대비 17.2% 줄었다. 패션업계에서는 제일모직과 신세계인터내셔날이 수익성 개선에 본격적으로 나섰다. 제일모직은 제조·직매형(SPA·패스트패션) 브랜드 ‘에잇세컨즈’와 아웃도어 브랜드 

In [11]:
train_dataset = Dataset.from_list(train_dataset)
test_dataset = Dataset.from_list(test_dataset)

In [12]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.float16
)

In [13]:
model_id = 'Qwen/Qwen2.5-7B-Instruct'

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map='auto'
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]